## Выбираем лучшие настройки для тикеров

In [9]:
%reload_ext autoreload
%autoreload 2

In [57]:
from app.cache import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm, TestAccAlgorithm
from app.config import RunConfig, AccConfig

test_configs = [
    {
        'acc': AccConfig.from_repr_string('auto_05 [0] |u0.5 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤ F'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F'),
        ]
    },
    # {
    #     'acc': AccConfig.from_repr_string('auto_05 [0] |u0.5 d0|'),
    #     'bots':  [
    #         RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
    #     ]
    # },
]

def run_test(config):
    bot_alg_list = []
    for bot_cfg in config['bots']:
        bot_alg_list.append(TestAlgorithm(do_printing=False, config=bot_cfg, use_cache=True))
        
    test_acc_alg = TestAccAlgorithm(
        do_printing=False, 
        config=config['acc'],
        bot_alg_list=bot_alg_list,
        use_cache=True
    )
    
    return test_acc_alg.test(
        last_test_date='2024-10-24',
        test_days_num=2,
    )

# unique_configs = set(test_configs)
unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))

for cfg in test_configs:
    res = run_test(cfg)
    if res:
        results.append(res)
    progress.update_progress()

print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

for r in results:
    print(r['acc'])
    for bot in r['bots']:
        print(f"    profit={str(bot['profit_p']) + '%':<6} op={bot['op']:<4} \"{str(bot['config'])}\"")
# print(results)


# # Вывод результатов или их дальнейшая обработка
# sorted_results = sorted(results, key=lambda x: (x['config'].ticker, x['config'].pretest_type, -float(x['profit_p'])), reverse=False)
# 
# last_val = ''
# for item in sorted_results:
#     cur_val = item['config'].ticker
#     if cur_val != last_val:
#         last_val = cur_val
#         print()
#     print(item)

Запуск в 22:14
Закончено в 22:14, количество 1, длительность 0:00:03                                               
cache 122 vals, used 24 times
{'profit': -0.0, 'p_avg': -0.0, 'config': auto_05 [0] |u0.5 d0.0| }
    profit=0.02%  op=43   "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=0.36%  op=68   "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
    profit=-0.43% op=60   "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "


За 1 день 24 окт

    {'profit': 0.25, 'config': auto_05 [0] |u0.5 d0.0| }
        profit=0.24% op=20   "MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=0.3%  op=41   "RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F "
        profit=0.19% op=38   "MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F "


[[{'exp': 'MVID fan F', 'profit': 240, 'profit_p': 0.24, 'profit_p_avg': 0.24, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F , 'last_conf': MVID+ 10/fan7:0/2 x l48 x 0.6(+x0.2)¤ F , 'op': 20}, 

{'exp': 'RNFT fan F', 'profit': 303, 'profit_p': 0.3, 'profit_p_avg': 0.3, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ F , 'last_conf': RNFT+ 8/fan7:0/2 x l40 x 0.6(+x0.2)¤ F , 'op': 41}, 

{'exp': 'MTLR fan F', 'profit': 193, 'profit_p': 0.19, 'profit_p_avg': 0.19, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| F , 'last_conf': MTLR+ 10/fan7:0/2 x l48 x 0.4¤ |u0.01 d0.0| F , 'op': 38}]]

Целевые показатели
====

длительность 0:01:00                                               
cache 1502 vals, used 1568 times

[[{'exp': 'MVID fan ', 'profit': 1049, 'profit_p': 1.05, 'profit_p_avg': 0.04, 'config': MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': MVID+ 10/fan7:0/2 x l46 x 0.6(+x0.2)¤ , 'op': 285}, 

{'exp': 'RNFT fan ', 'profit': 1930, 'profit_p': 1.93, 'profit_p_avg': 0.06, 'config': RNFT+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤ , 'last_conf': RNFT+ 8/fan7:0/2 x l40 x 0.6(+x0.2)¤ , 'op': 426}, 

{'exp': 'MTLR fan ', 'profit': 131, 'profit_p': 0.13, 'profit_p_avg': 0.0, 'config': MTLR+ 4/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0| , 'last_conf': MTLR+ 10/fan7:0/2 x l46 x 0.4¤ |u0.01 d0.0| , 'op': 548}]]
